## DEF OF CLASSES "ROUTE" AND "AIRPORT"

In [1]:
class Airport:
    name = ""
    city = ""
    country = ""
    iata = ""
    icao = ""
    lat = 0.
    long = 0.
    def info(self):
        return f'{self.iata}, {self.icao}\n{self.city}, {self.country}\nlat={self.lat}, long={self.long}'

class Route:
    source_iata = ""
    source_icao = ""
    dest_iata = ""
    dest_icao = ""
    dest_city = ""
    source_city = ""
    dest_country = ""
    source_country = ""
    flux = 0 # passenger per year (2017)
    def info(self):
        return f'source: {self.source_city}, {self.source_country}\ndest: {self.dest_city}, {self.dest_country}\nflux: {self.flux}'
    
    # Nota : il flusso verrà preso da un file che contiene solo rotte
    # per numero di passeggeri maggiore di 50000 all'anno
    
    #aggiungo questa proprietà così che quando confronto due oggetti
    #di questa classe, controlla il contenuto degli oggetti, e non
    #l'hash. Serve per eliminare i duplicati.
    def __eq__(self, other) : 
        return self.__dict__ == other.__dict__

## LOAD DATABASES
Ho salvato i databases in formato ".npy" così non c'è bisogno di rieffettuare ogni volta tutta l'estrazione dai csv.

In [2]:
import pickle

try:
    with open("data/db_airports.dat", "rb") as f:
        airports = pickle.load(f)
except Exception as e: print(e)
try:
    with open("data/db_routes.dat", "rb") as f:
        routes = pickle.load(f)
except Exception as e: print(e)
try:    
    with open("data/db_allroutes.dat", "rb") as f:
        allroutes = pickle.load(f)
except Exception as e: print(e)

# Importo flussi dal file

### importo file dei flussi in una lista "airports_new"

In [3]:
import csv

i = 0
airports_new = []
with open('data/pdf_to_csv/passeggeri_voli_italiani_2017.csv') as file :
        reader = csv.reader(file, delimiter=',')
        for line in reader:
            print(line)
            airports_new.append(line)
            i += 1
print(i)

['partenza', 'arrivo', 'passeggeri']
['Catania', 'Roma Fiumicino', '1016262']
['Roma Fiumicino', 'Catania', '991516']
['Roma Fiumicino', 'Palermo', '799435']
['Palermo', 'Roma Fiumicino', '796016']
['Milano Linate', 'Roma Fiumicino', '597754']
['Roma Fiumicino', 'Milano Linate', '585999']
['Catania', 'Milano Malpensa', '477510']
['Milano Malpensa', 'Catania', '473521']
['Cagliari', 'Roma Fiumicino', '467938']
['Roma Fiumicino', 'Cagliari', '457099']
['Bari', 'Roma Fiumicino', '380980']
['Roma Fiumicino', 'Bari', '370653']
['Cagliari', 'Milano Linate', '315440']
['Milano Linate', 'Cagliari', '311859']
['Catania', 'Milano Linate', '295959']
['Milano Linate', 'Catania', '289850']
['Roma Fiumicino', 'Torino', '279460']
['Torino', 'Roma Fiumicino', '270784']
['Brindisi', 'Roma Fiumicino', '262211']
['Napoli', 'Milano Linate', '258545']
['Roma Fiumicino', 'Brindisi', '257239']
['Lamezia Terme', 'Roma Fiumicino', '256108']
['Milano Linate', 'Napoli', '250706']
['Venezia', 'Roma Fiumicino', '2

### traduco i nomi delle città per mantenere i nomi del nostro database iniziale. Inoltre modifico i nomi per città con più aeroporti, cosí da poterci agire singolarmente.

In [4]:
#aggiungo iata aeroporto di verona
it_airports = []
for airport in airports:
    if airport.country == 'Italy':
        it_airports.append(airport)
for airport in it_airports:
    if airport.name == 'Verona Villafranca Airport':
        airport.iata == 'VRN'

translated_airports_new = []
for airport in airports_new:
    translated_airport = []
    for i in range(len(airports_new[0])):
        if 'Napoli' in airport[i]:
            translated_airport.append(airport[i].replace('Napoli','Naples'))
        elif 'Roma Fiumicino' in airport[i]:
            translated_airport.append(airport[i].replace('Roma Fiumicino','Rome1'))
        elif 'Roma Ciampino' in airport[i]:
            translated_airport.append(airport[i].replace('Roma Ciampino','Rome2'))
        elif 'Milano Linate' in airport[i]:
            translated_airport.append(airport[i].replace('Milano Linate','Milan1'))
        elif 'Milano Malpensa' in airport[i]:
            translated_airport.append(airport[i].replace('Milano Malpensa','Milan2'))
        elif 'Venezia' in airport[i]:
            translated_airport.append(airport[i].replace('Venezia','Venice'))
        elif 'Lamezia Terme' in airport[i]:
            translated_airport.append(airport[i].replace('Lamezia Terme','Lamezia'))
        elif 'Genova' in airport[i]:
            translated_airport.append(airport[i].replace('Genova','Genoa'))
        elif 'Firenze' in airport[i]:
            translated_airport.append(airport[i].replace('Firenze','Florence'))
        elif 'Trieste' in airport[i]:
            translated_airport.append(airport[i].replace('Trieste','Ronchi De Legionari'))
        elif 'Verona' in airport[i]:
            translated_airport.append(airport[i].replace('Verona','Villafranca'))
        else:
            translated_airport.append(airport[i])

        
    translated_airports_new.append(translated_airport)

del translated_airports_new[0]

for airport_enac in translated_airports_new:
    for airport in it_airports:
        if airport_enac[0] == airport.city and airport.iata != '\\N':
            airport_enac.append(airport.iata)

for airport_enac in translated_airports_new:
    if airport_enac[0] == 'Rome1':
        airport_enac.append('FCO')
    if airport_enac[0] == 'Rome2':
        airport_enac.append('CIA')
    if airport_enac[0] == 'Milan1':
        airport_enac.append('LIN')
    if airport_enac[0] == 'Milan2':
        airport_enac.append('MXP')
            
for airport_enac in translated_airports_new:
    for airport in it_airports:
        if airport_enac[1] == airport.city and airport.iata != '\\N':
            airport_enac.append(airport.iata)

for airport_enac in translated_airports_new:
    if airport_enac[1] == 'Rome1':
        airport_enac.append('FCO')
    if airport_enac[1] == 'Rome2':
        airport_enac.append('CIA')
    if airport_enac[1] == 'Milan1':
        airport_enac.append('LIN')
    if airport_enac[1] == 'Milan2':
        airport_enac.append('MXP')

for line in translated_airports_new:
    print(line)

['Catania', 'Rome1', '1016262', 'CTA', 'FCO']
['Rome1', 'Catania', '991516', 'FCO', 'CTA']
['Rome1', 'Palermo', '799435', 'FCO', 'PMO']
['Palermo', 'Rome1', '796016', 'PMO', 'FCO']
['Milan1', 'Rome1', '597754', 'LIN', 'FCO']
['Rome1', 'Milan1', '585999', 'FCO', 'LIN']
['Catania', 'Milan2', '477510', 'CTA', 'MXP']
['Milan2', 'Catania', '473521', 'MXP', 'CTA']
['Cagliari', 'Rome1', '467938', 'CAG', 'FCO']
['Rome1', 'Cagliari', '457099', 'FCO', 'CAG']
['Bari', 'Rome1', '380980', 'BRI', 'FCO']
['Rome1', 'Bari', '370653', 'FCO', 'BRI']
['Cagliari', 'Milan1', '315440', 'CAG', 'LIN']
['Milan1', 'Cagliari', '311859', 'LIN', 'CAG']
['Catania', 'Milan1', '295959', 'CTA', 'LIN']
['Milan1', 'Catania', '289850', 'LIN', 'CTA']
['Rome1', 'Torino', '279460', 'FCO', 'TRN']
['Torino', 'Rome1', '270784', 'TRN', 'FCO']
['Brindisi', 'Rome1', '262211', 'BDS', 'FCO']
['Naples', 'Milan1', '258545', 'NAP', 'LIN']
['Rome1', 'Brindisi', '257239', 'FCO', 'BDS']
['Lamezia', 'Rome1', '256108', 'SUF', 'FCO']
['Milan

### controllo che tutti gli aeroporti nel nuovo file siano già nel nostro database

In [5]:
i = 0
for line in translated_airports_new:
    if not line[3] in [airport.iata for airport in airports] or not line[4] in [airport.iata for airport in airports]:
        print(line)
        i += 1
if i == 0: print('OK!')

OK!


# Aggiungo l'info sul flusso agli oggetti nella lista it_routes

In [6]:
it_routes = []
for route in routes:
    if route.dest_country == 'Italy' and route.source_country == 'Italy':
        it_routes.append(route)

for route in it_routes: route.flux = 0 # inizializzo a zero tutti i flussi

for line in translated_airports_new:
    for route in it_routes:
        if line[3]==route.source_iata and line[4]==route.dest_iata :
            route.flux += int(line[2])

it_routes = sorted(it_routes, key=lambda x: -x.flux)
#for route in it_routes:
#    route.info()

i = 0
for route in it_routes:
    if route.flux == 0:
        i += 1
        route.flux += 1
print('rotte con flusso zero',i)
print('tutte le rotte',len(it_routes))

rotte con flusso zero 144
tutte le rotte 302


In [7]:
import folium

useless_airports = []
for airport in it_airports:
    if (airport.iata not in [route.dest_iata for route in it_routes]) and (airport.iata not in [route.source_iata for route in it_routes]):
        useless_airports.append([airport.iata, airport.city])

def create_map(center=[42,13]):
    return folium.Map(location=center, zoom_start=5)

def add_airport(mappa,airport):
    if airport.iata in [useless_airport[0] for useless_airport in useless_airports]:
        folium.Circle(radius=5e1, location=[airport.lat, airport.long], popup=airport.info(), color='white'
                      ).add_to(mappa)
    else:
        a = folium.Circle(radius=5e3, location=[airport.lat, airport.long], popup=airport.info() 
                         ).add_to(mappa)

def add_link(mappa, route, airports):
    for airport in airports:
        if route.source_iata == airport.iata:
            source_lat = airport.lat
            source_long = airport.long
        elif route.source_icao == airport.icao:
            source_lat = airport.lat
            source_long = airport.long
        elif route.dest_iata == airport.iata:
            dest_lat = airport.lat
            dest_long = airport.long
        elif route.dest_icao == airport.icao:
            dest_lat = airport.lat
            dest_long = airport.long
        #else: print('Problem')
    folium.PolyLine(locations=[[source_lat, source_long], [dest_lat, dest_long]],
                    weight=1e-6*route.flux, color='black', popup=route.info()
                   ).add_to(mappa)
    
    
m = create_map()
for airport in it_airports:
    add_airport(m, airport)

for route in it_routes:
    add_link(m, route, it_airports)
m.save('mappa_italia_tutti_aeroporti.html')
m

# Calcolo probabilità per ogni aeroporto di essere contagiato dai suoi primi vicini:

$P_{A \to B}$:= probabilità per $B$ di essere contagiato da $A$:
$$P_{A \to B} = \frac{F_{A \to B}}{\sum_{C \in V_B}F_{C \to B}}$$
dove $V_B$ sono i primi vicini di $B$ e $F_{A \to B}$ è il flusso di passeggeri da $A$ a $B$.

### In ogni aeroporto abbiamo aggiunto un attributo di nome "neigs" che è una lista di liste. Ognuna di esse rappresenta un link che arriva all'aeroporto in qustione ed ha i seguenti elementi:
- oggetto route della tratta in questione
- totale flussi che arrivano nell'aeroporto in questione
- probabilità definita sopra.

In [8]:
for airport in it_airports:
    airport.neigs_in = []
    for route in it_routes:
        if airport.iata == route.dest_iata:
            airport.neigs_in.append([route])
for airport in it_airports:
    totale = 0
    for route in airport.neigs_in:
        totale += route[0].flux
    for route in airport.neigs_in:
        if len(airport.neigs_in)!=0 : route.append(totale)

for airport in it_airports:
    for neig in airport.neigs_in:
        neig.append(neig[0].flux/neig[1])

### Controllo che la somma delle probabilità faccia uno per ogni aeroporto.

In [9]:
ii = 0
for airport in it_airports:
    totale = 0.
    for neig in airport.neigs_in:
        totale += neig[2]
    #print(totale)
    if totale == 0.0:
        ii += 1
print(ii)

len(useless_airports)

49


49

In [10]:
def fnc(x):
    try: return -x.neigs_in[0][1]
    except: return 0
    
print('*** CITTÀ, IATA, TOT FLUX ENTRANTE')
for airport in sorted(it_airports, key=fnc):
    try: print(airport.city, airport.iata, airport.neigs_in[0][1])
    except: print(airport.city, airport.iata, 0.)

*** CITTÀ, IATA, TOT FLUX ENTRANTE
Rome FCO 5583423
Catania CTA 2828829
Milan LIN 2405294
Palermo PMO 2048911
Milano MXP 1434120
Bergamo BGY 1431566
Cagliari CAG 1347056
Bari BRI 1229534
Naples NAP 1063823
Lamezia SUF 953377
Bologna BLQ 908467
Torino TRN 888458
Brindisi BDS 790186
Pisa PSA 634670
Venice VCE 567109
Olbia OLB 530278
Alghero AHO 387996
Villafranca VRN 372713
Treviso TSF 306861
Trapani TPS 228460
Reggio Calabria REG 175707
Genoa GOA 175404
Ronchi De Legionari TRS 132596
Florence FLR 119504
Pescara PSR 93486
Rome CIA 76882
Ancona AOI 4
Lampedusa LMP 3
Pantelleria PNL 3
Parma PMF 3
Cuneo CUF 3
Comiso CIY 3
Perugia PEG 2
Bolzano BZO 1
Amendola \N 0.0
Crotone CRV 0.0
Foggia FOG 0.0
Grottaglie TAR 0.0
Lecce LCC 0.0
Gioia Del Colle \N 0.0
Palermo \N 0.0
Sigonella NSY 0.0
Decimomannu DCI 0.0
Tortoli TTB 0.0
Turin \N 0.0
Milano \N 0.0
Albenga ALL 0.0
Cameri \N 0.0
Piacenza \N 0.0
Aviano AVB 0.0
Cervia \N 0.0
Rivolto \N 0.0
Forli FRL 0.0
Ghedi \N 0.0
Verona \N 0.0
Brescia VBS 0.0
R

## Ora facciamo una mappa in cui possiamo visualizzare un aeroporto e i suoi prossimi vicini entranti. Ogni link sarà pesato in base alla probabilità calcolata sopra.

In [11]:
import folium

useless_airports = []
for airport in it_airports:
    if (airport.iata not in [route.dest_iata for route in it_routes]) and (airport.iata not in [route.source_iata for route in it_routes]):
        useless_airports.append([airport.iata, airport.city])

def create_map(center=[42,13]):
    return folium.Map(location=center, zoom_start=5)

def get_lat_long(route, airports):
    for airport in airports:
        if route.source_iata == airport.iata:
            source_lat = airport.lat
            source_long = airport.long
            break
        elif route.source_icao == airport.icao:
            source_lat = airport.lat
            source_long = airport.long
            break
    for airport in airports:
        if route.dest_iata == airport.iata:
            dest_lat = airport.lat
            dest_long = airport.long
            break
        elif route.dest_icao == airport.icao:
            dest_lat = airport.lat
            dest_long = airport.long
            break
    return [source_lat, source_long], [dest_lat, dest_long]
        
def add_link(mappa, neig, airports):
    route = neig[0]
    source_coord, dest_coord = get_lat_long(route, airports)

    folium.PolyLine(locations=[source_coord, dest_coord],
                    weight=10.*neig[2], color='black', tooltip=f'{neig[0].source_city}\n{neig[2]:.2f}'
                   ).add_to(mappa)
    for airport in airports:
        if airport.iata == neig[0].dest_iata:
            folium.Circle(radius=5e3, location=[airport.lat, airport.long], popup=airport.info() 
                     ).add_to(mappa)
            

def add_airport_groups(mappa, airport, airports):
    if airport.iata in [useless_airport[0] for useless_airport in useless_airports]:
        
        folium.Circle(radius=5e3, location=[airport.lat, airport.long], popup=airport.info(), color='green'
                      ).add_to(g_senza_coll)
        
    else:
        g = folium.FeatureGroup(airport.city+', '+airport.iata, show=False)
        
        folium.Circle(radius=5e3, location=[airport.lat, airport.long], popup=airport.info()
                      ).add_to(g)
        
        for neig in airport.neigs_in :
            add_link(g, neig, airports)
        
        mappa.add_child(g)


g_senza_coll = folium.FeatureGroup('Senza collegamenti', show=False)
mm = create_map()

for airport in sorted(it_airports, key=fnc):
    add_airport_groups(mm, airport, it_airports)

mm.add_child(g_senza_coll)
    
folium.LayerControl().add_to(mm)
mm.save('mappa_italia_prossimi_vicini_in.html')
mm

# Calcolo probabilità per ogni aeroporto di contagiare i suoi primi vicini:

$P_{A \to B}$:= probabilità per A di influenzare B:
$$P_{A \to B} = \frac{F_{A \to B}}{\sum_{C \in V_A}F_{A \to C}}$$
dove $V_A$ sono i primi vicini di $A$ e $F_{A \to B}$ è il flusso di passeggeri da $A$ a $B$.

### In ogni aeroporto abbiamo aggiunto un attributo di nome "neigs" che è una lista di liste. Ognuna di esse rappresenta un link che arriva all'aeroporto in qustione ed ha i seguenti elementi:
- oggetto route della tratta in questione
- totale flussi che arrivano nell'aeroporto in questione
- probabilità definita sopra.

In [13]:
for airport in it_airports:
    airport.neigs_out = []
    for route in it_routes:
        if airport.iata == route.source_iata:
            airport.neigs_out.append([route])
for airport in it_airports:
    totale = 0
    for route in airport.neigs_out:
        totale += route[0].flux
    for route in airport.neigs_out:
        if len(airport.neigs_out)!=0 : route.append(totale)

for airport in it_airports:
    for neig in airport.neigs_out:
        neig.append(neig[0].flux/neig[1])

### Controllo che la somma delle probabilità faccia uno per ogni aeroporto.

In [14]:
ii = 0
for airport in it_airports:
    totale = 0.
    for neig in airport.neigs_out:
        totale += neig[2]
    #print(totale)
    if totale == 0.0:
        ii += 1
print(ii)

len(useless_airports)

49


49

In [15]:
def fnc(x):
    try: return -x.neigs_out[0][1]
    except: return 0
    
print('*** CITTÀ, IATA, TOT FLUX USCENTE')
for airport in sorted(it_airports, key=fnc):
    try: print(airport.city, airport.iata, airport.neigs_out[0][1])
    except: print(airport.city, airport.iata, 0.)

*** CITTÀ, IATA, TOT FLUX USCENTE
Rome FCO 5496557
Catania CTA 2863476
Milan LIN 2384823
Palermo PMO 2042255
Milano MXP 1433583
Bergamo BGY 1418540
Cagliari CAG 1359579
Bari BRI 1239484
Naples NAP 1089479
Lamezia SUF 962965
Bologna BLQ 902206
Torino TRN 880945
Brindisi BDS 802499
Pisa PSA 642455
Venice VCE 568581
Olbia OLB 530903
Alghero AHO 392688
Villafranca VRN 372443
Treviso TSF 306479
Trapani TPS 229208
Reggio Calabria REG 189133
Genoa GOA 171462
Ronchi De Legionari TRS 131183
Florence FLR 128002
Pescara PSR 97102
Rome CIA 78680
Ancona AOI 4
Lampedusa LMP 3
Pantelleria PNL 3
Parma PMF 3
Cuneo CUF 3
Comiso CIY 3
Perugia PEG 2
Bolzano BZO 1
Amendola \N 0.0
Crotone CRV 0.0
Foggia FOG 0.0
Grottaglie TAR 0.0
Lecce LCC 0.0
Gioia Del Colle \N 0.0
Palermo \N 0.0
Sigonella NSY 0.0
Decimomannu DCI 0.0
Tortoli TTB 0.0
Turin \N 0.0
Milano \N 0.0
Albenga ALL 0.0
Cameri \N 0.0
Piacenza \N 0.0
Aviano AVB 0.0
Cervia \N 0.0
Rivolto \N 0.0
Forli FRL 0.0
Ghedi \N 0.0
Verona \N 0.0
Brescia VBS 0.0
Ri

## Ora facciamo una mappa in cui possiamo visualizzare un aeroporto per volta e i suoi prossimi vicini uscenti. Ogni link sarà pesato in base alla probabilità calcolata sopra.

In [16]:
import folium

import random
def random_color():
    r = lambda: random.randint(0, 150)
    g = lambda: random.randint(0,255)
    b = lambda: random.randint(0, 150)

    return '#%02X%02X%02X' % (r(),g(),b())
colors = []

for ii in range(len(it_airports)):
    colors.append(random_color())

useless_airports = []
for airport in it_airports:
    if (airport.iata not in [route.dest_iata for route in it_routes]) and (airport.iata not in [route.source_iata for route in it_routes]):
        useless_airports.append([airport.iata, airport.city])

def create_map(center=[42,13]):
    return folium.Map(location=center, zoom_start=5)

def get_lat_long(route, airports):
    for airport in airports:
        if route.source_iata == airport.iata:
            source_lat = airport.lat
            source_long = airport.long
            break
        elif route.source_icao == airport.icao:
            source_lat = airport.lat
            source_long = airport.long
            break
    for airport in airports:
        if route.dest_iata == airport.iata:
            dest_lat = airport.lat
            dest_long = airport.long
            break
        elif route.dest_icao == airport.icao:
            dest_lat = airport.lat
            dest_long = airport.long
            break
    return [source_lat, source_long], [dest_lat, dest_long]
        
def add_link(mappa, neig, airports, color):
    route = neig[0]
    source_coord, dest_coord = get_lat_long(route, airports)

    folium.PolyLine(locations=[source_coord, dest_coord],
                    weight=10.*neig[2], color=color, tooltip=f'{neig[0].dest_city}\n{neig[2]:.2f}'
                   ).add_to(mappa)
    for airport in airports:
        if airport.iata == neig[0].dest_iata:
            folium.Circle(radius=5e3, location=[airport.lat, airport.long], popup=airport.info() 
                     ).add_to(mappa)
            

def add_airport_groups(mappa, airport, airports, color):
    if airport.iata in [useless_airport[0] for useless_airport in useless_airports]:
        
        folium.Circle(radius=5e3, location=[airport.lat, airport.long], popup=airport.info(), color='green'
                      ).add_to(g_senza_coll)
        
    else:
        g = folium.FeatureGroup(airport.city+', '+airport.iata, show=False)
        
        folium.Circle(radius=5e3, location=[airport.lat, airport.long], popup=airport.info()
                      ).add_to(g)
        
        for neig in airport.neigs_out :
            add_link(g, neig, airports, color)
        
        mappa.add_child(g)


g_senza_coll = folium.FeatureGroup('Senza collegamenti', show=False)
mm = create_map()

for ii, airport in enumerate(sorted(it_airports, key=fnc)):
    add_airport_groups(mm, airport, it_airports, colors[ii])

    
mm.add_child(g_senza_coll)
    
folium.LayerControl().add_to(mm)
mm.save('mappa_italia_primi_vicini_out.html')
mm

In [17]:
# # to save database:
# import pickle 
# with open("data/db_routes_new.dat", 'wb') as f:
#     pickle.dump(routes, f)